# Sentiment Analysis - Positive / Negative

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import json
import numpy as np
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Input
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from IPython.display import clear_output

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
# loading in the 13 CSVs
df1 = pd.read_csv('./russian-troll-tweets/IRAhandle_tweets_1.csv')
df2 = pd.read_csv('./russian-troll-tweets/IRAhandle_tweets_2.csv')
df3 = pd.read_csv('./russian-troll-tweets/IRAhandle_tweets_3.csv')
df4 = pd.read_csv('./russian-troll-tweets/IRAhandle_tweets_4.csv')
df5 = pd.read_csv('./russian-troll-tweets/IRAhandle_tweets_5.csv')
df6 = pd.read_csv('./russian-troll-tweets/IRAhandle_tweets_6.csv')
df7 = pd.read_csv('./russian-troll-tweets/IRAhandle_tweets_7.csv')
df8 = pd.read_csv('./russian-troll-tweets/IRAhandle_tweets_8.csv')
df9 = pd.read_csv('./russian-troll-tweets/IRAhandle_tweets_9.csv')
df10 = pd.read_csv('./russian-troll-tweets/IRAhandle_tweets_10.csv')
df11 = pd.read_csv('./russian-troll-tweets/IRAhandle_tweets_11.csv')
df12 = pd.read_csv('./russian-troll-tweets/IRAhandle_tweets_12.csv')
df13 = pd.read_csv('./russian-troll-tweets/IRAhandle_tweets_13.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0,15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# going to combine these dataframes together for ease of use
frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13]

In [5]:
master = pd.concat(frames)

In [6]:
# Dealing with non-English tweets, while interesting, is beyond my linguistic capabilities!
mask = master['language'] == 'English'
eng = master[mask]

### Building a Postive / Negative Sentiment NN Classifier

In [8]:
# extract data from a csv
# notice the cool options to skip lines at the beginning
# and to only take data from certain columns
training = np.genfromtxt('./Sentiment Analysis Dataset.csv', delimiter=',', skip_header=1, usecols=(1, 3), dtype=None)

# create our training data from the tweets
train_x = [x[1] for x in training]
# index all the sentiment labels
train_y = np.asarray([x[0] for x in training])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  after removing the cwd from sys.path.


In [9]:
# only work with the 3000 most popular words found in our dataset
max_words = 3000

# create a new Tokenizer
tokenizer = Tokenizer(num_words=max_words)
# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(train_x)

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index
# Let's save this out so we can use it later
with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)


def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

allWordIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
allWordIndices = np.asarray(allWordIndices)

# create one-hot matrices out of the indexed tweets
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
# treat the labels as categories
train_y = keras.utils.to_categorical(train_y, 2)

In [11]:
model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [12]:
model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

### Training Network

In [15]:
# commenting out, so I don't accidently run!
# model.fit(train_x, train_y, batch_size=32, epochs=5,verbose=1, validation_split=0.1, shuffle=True)

Train on 1420764 samples, validate on 157863 samples
Epoch 1/5
1420764/1420764 [==============================] - 1333s 938us/step - loss: 0.4949 - acc: 0.7609 - val_loss: 0.4493 - val_acc: 0.7891
Epoch 2/5
1420764/1420764 [==============================] - 985s 693us/step - loss: 0.4739 - acc: 0.7761 - val_loss: 0.4441 - val_acc: 0.7926
Epoch 3/5
1420764/1420764 [==============================] - 1059s 746us/step - loss: 0.4659 - acc: 0.7821 - val_loss: 0.4449 - val_acc: 0.7917
Epoch 4/5
1420764/1420764 [==============================] - 1124s 791us/step - loss: 0.4608 - acc: 0.7861 - val_loss: 0.4491 - val_acc: 0.7918
Epoch 5/5
1420764/1420764 [==============================] - 1050s 739us/step - loss: 0.4567 - acc: 0.7886 - val_loss: 0.4441 - val_acc: 0.7927


### Saving the Model

In [16]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')

In [13]:
model.load_weights('model.h5')

### Loading in Troll Tweets

In [14]:
content_cat_ddf = eng[['content', 'account_category']]

In [15]:
%%time
content_cat_df = content_cat_ddf.sample(frac=0.2).drop_duplicates()
print(content_cat_df.shape[0], 'tweets loaded')

395496 tweets loaded
CPU times: user 644 ms, sys: 77.3 ms, total: 721 ms
Wall time: 781 ms


### Running IRA Tweets through Sentiment Analysis Model

In [16]:
# test data
maxlen=3000
list_tweets = content_cat_df["content"].fillna("Invalid").values
list_tokenized_tweets = tokenizer.texts_to_sequences(list_tweets)
X_twe = sequence.pad_sequences(list_tokenized_tweets, maxlen=maxlen)

In [23]:
%%time
y_twe = model.predict(X_twe, batch_size=1024, verbose=True)

395496/395496 [==============================] - 32s 80us/step
CPU times: user 1min 1s, sys: 5.86 s, total: 1min 7s
Wall time: 31.8 s


### Interpreting Results

In [18]:
labels = ['positive', 'negative']

In [20]:
from IPython.display import Markdown, display
display_markdown = lambda x: display(Markdown(x))
def show_sentence(sent_idx):
    display_markdown('# Input Sentence:\n `{}`'.format(list_tweets[sent_idx]))
    c_pred = model.predict(X_twe[sent_idx:sent_idx+1])[0]
    display_markdown('## Scores')
    for k, p in zip(labels, c_pred):
        display_markdown('- {}, Prediction: {:2.2f}%'.format(k, 100*p))
show_sentence(np.random.randint(0,1000))

# Input Sentence:
 `#MyOlympicSportWouldBe the 3-legged race because I'm, well... shall I say gifted`

## Scores

- positive, Prediction: 44.55%

- negative, Prediction: 55.45%

In [24]:
senti_df = pd.DataFrame(y_twe, columns = labels)
senti_df['content_category'] = content_cat_df['account_category'].values.copy()

In [68]:
senti_df.sample(5)

,negative,positive,content_category
282900,0.509092,0.490908,RightTroll
139275,0.466040,0.533960,NewsFeed
6540,0.481034,0.518966,Commercial
385136,0.445500,0.554500,NewsFeed
191505,0.408626,0.591374,RightTroll


In [25]:
worst_tweets = np.argsort(senti_df['negative'].values)
for _, idx in zip(range(5), 
                  worst_tweets):
    show_sentence(idx)

# Input Sentence:
 `RT entjboss: I really want to believe in Mitch, but I can't tolerate excuses. We have the capability to have a HUG… https://t.co/DFy5N0LPj4`

## Scores

- positive, Prediction: 82.77%

- negative, Prediction: 17.23%

# Input Sentence:
 `RT ShaunLeaneMP: There can be no doubt now the Victorian Liberals have put in a heap of time and effort to secure the gangster vote …`

...

- negative, Prediction: 21.16%

# Input Sentence:
 `RT CollinRugg: Wasserman Shultz, Imran Awan &amp; a Senator saying "I Hope Trump Is Assassinated"  Not a good day to be a Dem. Way to keep it c…`

## Scores

- positive, Prediction: 78.58%

- negative, Prediction: 21.42%

In [26]:
best_tweets = np.argsort(senti_df['positive'].values)
for _, idx in zip(range(5), 
                  best_tweets):
    show_sentence(idx)

# Input Sentence:
 `My wife is a social butterfly.  Me ?   I'm in the cocoon stage... waiting for my metamorphosis.  Just a hunch, but it may never come.`

## Scores

- positive, Prediction: 3.28%

- negative, Prediction: 96.72%

# Input Sentence:
 `#top RT KORANISBURNING: I see is Trump is setting up a chest board, if a #2ndSpecialCounsel is appointed This is about to get very messy re…`

...

- negative, Prediction: 93.76%

# Input Sentence:
 `Trump gives an interview to some Christian group revealing the intent of the ban is to help christians. That's not very Christian, right?`

## Scores

- positive, Prediction: 6.91%

- negative, Prediction: 93.09%